# CosmosDB - PostGres + Azure Cognitive Search
This sample shows how to create and use search index on Azure Cognitive Search
### Prerequisite: install python libraries
- Please make sure all the libraries found in requirements.txt are installed in your python environment. 
- Rename example.env to llm.env and enter your credentials in llm.env
- Whitelist your IP to access you CosmosDB - PostGres Cluster. Add you IP in "Networking" section of your PostGRes cluster on the [Azure Portal](portal.azure.com)

# Load environment variables and keys 

In [ ]:
from dotenv import dotenv_values

# specify the name of the .env file name 
env_name = "llm.env" # change to your own .env file name
config = dotenv_values(env_name)

# Connect to Cosmosdb Postgres (PG) 

In [ ]:
import psycopg2
from psycopg2 import pool

host = config["host"]
dbname = config["dbname"] 
user = config["user"] 
password = config["password"] 
sslmode = config["sslmode"] 

# Build a connection string from the variables
conn_string = "host={0} user={1} dbname={2} password={3} sslmode={4}".format(host, user, dbname, password, sslmode)

postgreSQL_pool = psycopg2.pool.SimpleConnectionPool(1, 20,conn_string)
if (postgreSQL_pool):
    print("Connection pool created successfully")

# Use getconn() to get a connection from the connection pool
connection = postgreSQL_pool.getconn()
cursor = connection.cursor()

In [ ]:
# Create a cursor object to interact with the database
cursor = connection.cursor()

# Execute the SQL query to retrieve the table names
cursor.execute("SELECT table_name FROM information_schema.tables WHERE table_schema='public'")

# Fetch all the results
table_names = cursor.fetchall()

# Print the table names
for table in table_names:
    print(table[0])

# Close the cursor and the database connection
cursor.close()
connection.close()

#### Load data to a table in the database
If this is the first time you are running the notebook, you need to load our sample dataset into the database first. We will create a new table "food_reviews" and load the data from the csv file.

In [ ]:
import pandas as pd
import numpy as np
from psycopg2 import Error
from psycopg2 import sql


# Read data into a DataFrame
df = pd.read_csv('../../DataSet/Reviews_small.csv')

# Specify the batch size
batch_size = 25000

# Define the table name
table_name = 'food_reviews'

# # Drop previous table of same name if one exists
# cursor.execute(f"DROP TABLE IF EXISTS {table_name};")
# print("Finished dropping table (if existed)")
# connection.commit()

# Execute the query to check if the table exists
cursor.execute(f"SELECT EXISTS (SELECT 1 FROM information_schema.tables WHERE table_name = '{table_name}');")

# Fetch the result
exists = cursor.fetchone()[0]

if exists:
    print(f"The table '{table_name}' exists in the database.")
    print("You may drop previous table (see commented code above) if you want to re-insert reviews.")
else:
    print(f"The table '{table_name}' does not exist in the database.Creating it now and inserting reviews ...")

    # Build a connection string from the variables
    conn_string = "host={0} user={1} dbname={2} password={3} sslmode={4}".format(host, user, dbname, password, sslmode)

    postgreSQL_pool = psycopg2.pool.SimpleConnectionPool(1, 20, conn_string)
    if postgreSQL_pool:
        print("Connection pool created successfully")

    # Use getconn() to get a connection from the connection pool
    with postgreSQL_pool.getconn() as connection:

        # Define the CREATE TABLE query
        create_table_query = f"""
        CREATE TABLE IF NOT EXISTS {table_name} (
            Id INTEGER PRIMARY KEY,
            ProductId TEXT,
            UserId TEXT,
            ProfileName TEXT,
            HelpfulnessNumerator INTEGER,
            HelpfulnessDenominator INTEGER,
            Score INTEGER,
            Time INTEGER,
            Summary TEXT,
            Text TEXT
        );
        """

        # Execute the CREATE TABLE query
        try:
            with connection.cursor() as cursor:
                cursor.execute(create_table_query)
                connection.commit()
                print(f"Table {table_name} created successfully!")
        except (Exception, Error) as e:
            print(f"Error creating table {table_name}: {e}")
            connection.rollback()

        # Assuming your DataFrame variable is called 'df'
        # Convert DataFrame to a list of tuples for bulk insertion
        records = df.to_records(index=False)
        records_list = records.tolist()

        # Convert numpy.int32 to int in each row
        records_list = [
            tuple(int(value) if isinstance(value, np.int32) else value for value in record)
            for record in records_list
        ]

        # Split the records list into batches
        batches = [records_list[i: i + batch_size] for i in range(0, len(records_list), batch_size)]

        # Iterate over each batch and perform bulk insert
        count = 0
        for batch in batches:
            count += 1
            print(f"Inserting batch {count} into the table")
            try:
                insert_query = sql.SQL(f"INSERT INTO {table_name} (Id, ProductId, UserId, ProfileName, HelpfulnessNumerator, HelpfulnessDenominator, Score, Time, Summary, Text) " \
                                    f"VALUES ({', '.join(['%s'] * len(batch[0]))})")
                
                with connection.cursor() as cursor:
                    cursor.executemany(insert_query, batch)
                    connection.commit()
            except (Exception, Error) as e:
                print(f"Error inserting batch into the table: {e}")
                connection.rollback()



#### Example query

In [ ]:
# Assuming you have already established a connection and have a cursor object

# Rollback the current transaction
connection.rollback()
cursor = connection.cursor()
# Execute the SELECT statement
try:
    cursor.execute(f"SELECT count(Id) FROM {table_name};")
    rows = cursor.fetchall()
    for row in rows:
        print(row)
except (Exception, Error) as e:
    print(f"Error executing SELECT statement: {e}")

## Retrieve data from database and store the embedding in CogSearch 
In this step, we will retrieve the id and concatenated data of desired columns from database first. Then we will use azure open ai to get the text embedding. We will then store the text embedding in azure CogSearch for the future retrieval purposes. 

#### Retrieve data from database

In [ ]:
# Assuming you have already established a connection and have a cursor object

# Rollback the current transaction
connection.rollback()

# Execute the SELECT statement
try:
    cursor.execute(f"SELECT id, CONCAT('productid: ', productid, ' ', 'score: ', score, ' ', 'text: ', text) AS concat FROM {table_name};")
    rows = cursor.fetchall()
except (Exception, Error) as e:
    print(f"Error executing SELECT statement: {e}")

#### Create the content and generate the embedding, store in a dataframe

In [ ]:
import openai
import pandas as pd

openai.api_type = config["openai_api_type"] 
openai.api_key = config['openai_api_key']
openai.api_base = config['openai_api_base'] 
openai.api_version = config['openai_api_version'] 


def createEmbeddings(text):
    response = openai.Embedding.create(input=text , engine=config["openai_deployment_embedding"])
    embeddings = response['data'][0]['embedding']
    return embeddings

content_embeddings = []
idx = []
for row in rows:
    idx.append(row[0])
    content_embeddings.append(createEmbeddings(row[1]))

# create a dataframe with the embeddings
df = pd.DataFrame({'embeddings': content_embeddings}, index=idx) 
df

#### Store the embeddings in Azure Cognitive Search Vector Store

[AzureCogSearch](https://learn.microsoft.com/en-us/azure/search/search-what-is-azure-search) provides a simple interface to create a vector database, store and retrieve data using vector search. You can read more about [here](https://github.com/Azure/cognitive-search-vector-pr/tree/main) more about Vector Search.

There are two steps to store data in AzureCogSearch vector database:
- First, we create the index (or schema) of the vector database
- Second, we add the chunked documents and their embeddings to the vector datastore

In [ ]:
import requests
import json

# Azure Cognitive Search
cogsearch_name = config["cogsearch_name"] #TODO: fill in your cognitive search name
cogsearch_index_name = config["cogsearch_index_name"] #TODO: fill in your index name: must only contain lowercase, numbers, and dashes
cogsearch_api_key = config["cogsearch_api_key"] #TODO: fill in your api key with admin key

EMBEDDING_LENGTH = 1536

### Create search index

In [ ]:
# Create Index for Cog Search with fields as id,  and contentVector
# Note the datatypes for each field below

url = f"https://{cogsearch_name}.search.windows.net/indexes/{cogsearch_index_name}?api-version=2023-07-01-Preview"
payload = json.dumps({
  "name": cogsearch_index_name,
  "fields": [
    {
      "name": "id",
      "type": "Edm.String",
      "key": True,
      "filterable": True
    },
    {
      "name": "contentVector",
      "type": "Collection(Edm.Single)",
      "searchable": True,
      "retrievable": True,
      "dimensions": EMBEDDING_LENGTH,
      "vectorSearchConfiguration": "vectorConfig"
    }
  ],
  "vectorSearch": {
    "algorithmConfigurations": [
      {
        "name": "vectorConfig",
        "kind": "hnsw",
        # "hnswParameters": {
        #   "m": 4,
        #   "efConstruction": 400,
        #   "metric": "cosine"
        # }
      }
    ]
  },
  "semantic": {
    "configurations": [
      {
        "name": "my-semantic-config",
        "prioritizedFields": {
          "prioritizedContentFields": [
            {
              "fieldName": "id"
            }
          ],
        }
      }
    ]
  }
})
headers = {
  'Content-Type': 'application/json',
  'api-key': cogsearch_api_key
}

response = requests.request("PUT", url, headers=headers, data=payload)
print(response.status_code)

In [ ]:
def batch_append_payload(df):
    """append payload for batch insertion (note: max 1000 rows per insertion) of embeddings to Cognitive Search"""
    value_list = []
    for index, row in df.iterrows():
        value_list.append(
            {
            "id": str(index),
            "contentVector": row['embeddings'],
            "@search.action": "upload"
            }
        )
    print('payload of size {}'.format(len(value_list)))
    print('start: {}'.format(value_list[0]))
    print('end: {}'.format(value_list[-1]))
    payload = json.dumps({
        "value": value_list
    })
    return payload

def BatchInsertToCogSearch(df): 
    """Batch insert of embeddings to Cognitive Search, note: column name must be 'embeddings'"""
    url = f"https://{cogsearch_name}.search.windows.net/indexes/{cogsearch_index_name}/docs/index?api-version=2023-07-01-Preview"
    payload = batch_append_payload(df)
    headers = {
    'Content-Type': 'application/json',
    'api-key': cogsearch_api_key,
    }

    response = requests.request("POST", url, headers=headers, data=payload)
    print(response.json())

    if response.status_code == 200 or response.status_code == 201:
        return "Success"
    else:
        return "Failure"

In [ ]:
BatchInsertToCogSearch(df)

## User Asks a Question 
In this step, the code will convert the user's question to an embedding and then retieve the top K document chunks based on the users' question using the cosine similirity. Please note that other similarity metrics can also be used.

In [ ]:
userQuestion = "Great Taffy"
retrieve_k = 3 # Retrieve the top 2 documents from vector database

In [ ]:
# retrieve k chnuks
def retrieve_k_chunk(k, questionEmbedding):
    # Retrieve the top K entries
    url = f"https://{cogsearch_name}.search.windows.net/indexes/{cogsearch_index_name}/docs/search?api-version=2023-07-01-Preview"

    payload = json.dumps({
    "vector": {
        "value": questionEmbedding,
        "fields": "contentVector",
        "k": k
    }
    })
    headers = {
    'Content-Type': 'application/json',
    'api-key': cogsearch_api_key,
    }

    response = requests.request("POST", url, headers=headers, data=payload)
    output = json.loads(response.text)
    print(response.status_code)
    return output

# Generate embeddings for the question and retrieve the top k document chunks
questionEmbedding = createEmbeddings(userQuestion)
output = retrieve_k_chunk(retrieve_k, questionEmbedding)

In [ ]:
# Use the top k ids to retrieve the actual text from the database 
top_ids = []
for i in range(len(output['value'])):
    top_ids.append(int(output['value'][i]['id']))

print(top_ids)

#### Retrieve text from database

In [ ]:
# Assuming you have already established a connection and have a cursor object

# Rollback the current transaction
connection.rollback()

format_ids = ', '.join(['%s'] * len(top_ids))

sql = f"SELECT CONCAT('productid: ', productid, ' ', 'score: ', score, ' ', 'text: ', text) AS concat FROM {table_name} WHERE id IN ({format_ids})"

# Execute the SELECT statement
try:
    cursor.execute(sql, top_ids)    
    top_rows = cursor.fetchall()
    for row in top_rows:
        print(row)
except (Exception, Error) as e:
    print(f"Error executing SELECT statement: {e}")


# OPTIONAL: Offer Response to User's Question
In order to offer a response, a user can either follow a simple prompting method as shown below or leverage more sophisticated ways used by other libraries, such as [langchain](https://python.langchain.com/en/latest/index.html).

#### Prompting directly using Azure Open AI service

In [ ]:
# create a prompt template 
template = """
    context :{context}
    Answer the question based on the context above. Provide the product id associated with the answer as well. If the
    information to answer the question is not present in the given context then reply "I don't know".
    Question: {query}
    Answer: """

In [ ]:
# create the context from the top_rows
context = ""
for row in top_rows:
    context += row[0]
    context += "\n"
    
print(context)

In [ ]:
print(userQuestion)
prompt = template.format(context=context, query=userQuestion)
print(prompt)

In [ ]:

response = openai.Completion.create(
    engine= config["openai_deployment_completion"],
    prompt=prompt,
    max_tokens=1024,
    n=1,
    stop=None,
    temperature=1,
)

print("prompt: ", prompt)
print('~~~~~')
# print("response: ", response['choices'][0]['text'].replace('\n', '').replace(' .', '.').strip())
print(response['choices'][0]['text'])

